### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 2 - Digital Audio Processing 

In this assignment you will load, decode, and process digital audio files (e.g., MP3, WAV) using Python. For the following tasks, you will use our suggested libraries (see the setup section). For both audio formats you will extract and process content and some basic metadata. For the following tasks, you will use our suggested libraries (see the setup section). 

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments in Moodle.

If the cell below executes without error, you can start the assignment!

In [2]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
from enum import Enum
import mutagen      # mutagen
from mutagen.mp3 import MP3
from mutagen.id3 import ID3
from mutagen.easyid3 import EasyID3
import wave         # python's built-in wave library
import pandas as pd # pandas
import ffmpeg       # ffmpeg-python wrapper (requires ffmpeg.exe in your system path!)
import subprocess   # for calling local executables such as ffmpeg.exe


In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [3]:
# Please place your own imports here.
from mutagen import File
import os
from enum import Enum
import math

In [ ]:
# Place any helper functions, global variables and classes here.

## Task 2.1 Organize Audio files by specific criteria (35P):

In [3]:
# Write your function here.

#All the fields, both for sorting and getting the data
class Criteria(Enum):
    
    FILENAME = ("filename")
    FORMAT = ("format")
    ENCODER = ("tags", "TENC")
    DURATION = ("info", "length")
    ARTIST = ("tags", 'TPE1')
    TITLE = ("tags", "TIT2")
    DATE = ("tags", "TDRC")
    ALBUM = ("tags", 'TALB')
    TRACK = ("tags", "TRCK")
    COMPOSER = ("tags", "TCOM")
    GENRE = ("tags", 'TCON')
    SAMPLE_RATE = ("info", "sample_rate")
    BITRATE = ("info", "bitrate")
    CHANNELS = ("info", "channels")

    #Get the value of the field for the input file
    def get(self, file):
        try:
            if self.value[0] == "tags": 
                return File(file).tags[self.value[1]].text[0]
            elif self.value == "filename":
                return file.split("/")[-1]
            elif self.value == "format":
                return type(File(file)).__name__
            elif self.value[0] == "info":
                return getattr(File(file).info, self.value[1])
        except Exception as e: 
            return "Unknown"
            
    #Generates a pandas dataframe usign all the fields for the input file.
    @classmethod
    def generatePandas(_, file: str, index: int = 0):
        data = {}
        for x in Criteria:
            data[x.name.lower()] = x.get(file)
        return pd.DataFrame(data=data, index=[index])
        
    
# Auto-plays an audio file and also embeds an IPython audio display. <- This comment describes a completly different function
def MyAudioFilesOrganizer(inputDir: str, grouping: Criteria) -> pd.DataFrame:
    
    files = {}
    for file in os.listdir(inputDir): files[file] = grouping.get(inputDir+file)
    
    files = sorted(files.items(), key=lambda x:x[1])

    return pd.concat([Criteria.generatePandas(inputDir+file[0], i) for i, file in enumerate(files)])
    

In [6]:
# Test your function here.
#MyAudioFilesOrganizer("media/audio/", Criteria.DURATION)

## Task 2.2 Audio mixer (25P):

In [55]:
# Write your function here.

# Merges two audio files using FFMPEG.
def TwoAudioMixer(audioFile1: str, a1From: int, a1To: int, 
                  audioFile2: str, a2From: int, a2To: int, overlapDur: float, 
                  outputDir: str, outFilename: str) -> None:
    
    #               LOAD   FILE       START POINT                        CUT DURATION
    audio1 = ffmpeg.input(audioFile1, ss=a1From).filter('atrim', duration=a1To - a1From)
    
    #               LOAD   FILE       START POINT                        CUT DURATION            DELAY    DELAY PER CHANNEL (I CAN'T FIGURE OUT HOW TO DO IT TO ALL CHANNELS AT ONCE)
    audio2 = ffmpeg.input(audioFile2, ss=a2From).filter('atrim', duration=a2To - a2From).filter('adelay', "|".join([str(math.floor(((a1To - a1From) - overlapDur)*1000)) for i in range(ffmpeg.probe(audioFile1)["streams"][0]["channels"])]))
    
    #                    JOIN TRACKS                     OUTPUT DIRECTORY             MAKE THE FFMPEG CALLS
    ffmpeg.filter([audio1, audio2], 'amix').output(outputDir + "/" + outFilename).run(overwrite_output=True)

In [63]:
# Test your function here.
#TwoAudioMixer("media/audio/Huntza Lasai Lasai.mp3", 30, 70, "media/audio/Musikaren Doinua - ETS.mp3", 0, 120, 5, "media/audio", "mixed.mp3")

## Task 2.3 Concealing speakers ID by lowering/increasing the audio pitch (20P):

In [95]:
# Write your function here.

# Changes the pitch of an audio file using FFMPEG.
def VoicePitchChanger(audioFile: str, shift: float, outputDir: str, outFilename: str) -> None:
    realShift = 2**(shift/12) #From ChatGPT, 12 are the semitones per octave, each semitone doubles / halves the pitch of the previous.
    originalRate = int(ffmpeg.probe(audioFile)['streams'][0]['sample_rate'])
    output = outputDir + "/" + outFilename
    ffmpeg.input(audioFile)\
    .filter("asetrate", originalRate * realShift)\
    .filter("atempo", 1/realShift)\
    .output(output)\
    .run(overwrite_output=True)

In [96]:
# Test your function here.
""" factor = 5
audio = "Amazon.mp3"
ffmpeg.input("media/audio/"+audio)\
    .output("media/audio/generated/0original.mp3")\
    .run(overwrite_output=True)
VoicePitchChanger("media/audio/"+audio, -factor, "media/audio/generated", "1low.mp3")
VoicePitchChanger("media/audio/generated/1low.mp3", factor, "media/audio/generated", "2low_restored.mp3")
VoicePitchChanger("media/audio/"+audio, factor, "media/audio/generated", "3high.mp3")
VoicePitchChanger("media/audio/generated/3high.mp3", factor, "media/audio/generated", "4higher.mp3")
VoicePitchChanger("media/audio/"+audio, factor*2, "media/audio/generated", "5high2.mp3")
VoicePitchChanger("media/audio/generated/4higher.mp3", factor, "media/audio/generated", "6highest.mp3")
VoicePitchChanger("media/audio/generated/3high.mp3", -factor, "media/audio/generated", "7high_restored.mp3") """

{'index': 0, 'codec_name': 'mp3', 'codec_long_name': 'MP3 (MPEG audio layer 3)', 'codec_type': 'audio', 'codec_tag_string': '[0][0][0][0]', 'codec_tag': '0x0000', 'sample_fmt': 'fltp', 'sample_rate': '44100', 'channels': 2, 'channel_layout': 'stereo', 'bits_per_sample': 0, 'initial_padding': 0, 'r_frame_rate': '0/0', 'avg_frame_rate': '0/0', 'time_base': '1/14112000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 3622993517, 'duration': '256.731400', 'bit_rate': '160000', 'disposition': {'default': 0, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic': 0, 'timed_thumbnails': 0, 'non_diegetic': 0, 'captions': 0, 'descriptions': 0, 'metadata': 0, 'dependent': 0, 'still_image': 0}}
{'index': 0, 'codec_name': 'mp3', 'codec_long_name': 'MP3 (MPEG audio layer 3)', 'codec_type': 'audio', 'codec_tag_string': '[0][0][0][0]', 'codec_tag': '0x0000', 'sample_fmt': 'fltp', 'sample_